# Import

In [1]:
!pip install flask-ngrok

In [2]:
import numpy as np
import pandas as pd
import json
import ast
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

# Load data

In [8]:
with open('/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Recommend/svd_400_100_top_50.json', 'r') as f:
    top_50_animes = json.load(f)

In [9]:
rating = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Rating/enable_rating.csv')
anime = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Raw/anime.csv')
anime = anime[anime['anime_id'].isin(rating['anime_id'].unique())]
anime['genre'] = anime['genre'].fillna('')
anime['type'] = anime['type'].fillna('Unknown')

In [10]:
anime_ids = anime['anime_id'].unique()
user_ids = rating['user_id'].unique().tolist()

# Utilities

In [11]:
class Anime(object):
    def __init__(self, iid):
        object.__init__(self)
        row = anime[anime['anime_id'] == iid].iloc[0]
        self.anime_id = row['anime_id']
        self.name = row['name']
        genre = row['genre']
        if len(genre) > 50:
            self.genre = genre[:47] + '...'
        else:
            self.genre = genre
        self.type_ = row['type']
        self.episodes = row['episodes']
        self.members = row['members']

In [12]:
class User(object):
    def __init__(self, uid):
        object.__init__(self)
        self.rated_animes = [Anime(iid) for iid in top_50_animes[uid]['rated_animes']]
        self.recommended_animes = [Anime(iid) for iid in top_50_animes[uid]['recommeded_animes']]

# App

In [14]:
from flask import Flask
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
static_folder="/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Demo/static"
template_folder="/content/drive/MyDrive/Colab Notebooks/Anime Recommendation System/Demo/templates"
app = Flask(__name__, template_folder=template_folder, static_folder=static_folder)
run_with_ngrok(app)
@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        input_id = request.form['name']
        if int(input_id) not in user_ids:
            return render_template("index.html")
        else:
            user = User(str(int(input_id)))
            return render_template("result.html", user=user)
    return render_template("index.html")

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://35ddc1c2555c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
